In [1]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [2]:
X_train = X_train.reshape(-1, 784) 
X_test = X_test.reshape(-1, 784)
X_train = X_train / 255.
X_test = X_test / 255.

In [3]:
#학습용, 검증용 데이터를 텐서로 변환
import torch
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train.astype('int32')).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test.astype('int32')).long()
# 변환된 텐서의 샘플수
print(X_train.shape)
print(y_train.shape)

torch.Size([60000, 784])
torch.Size([60000])


In [4]:
#텐서를 gpu로 이동
X_train = X_train.cuda()
y_train = y_train.cuda()
X_test = X_test.cuda()
y_test = y_test.cuda()

In [5]:
#신경망 구성
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)
model = Net().cuda()

In [6]:
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(Net, max_epochs=20, lr=0.1)
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6469       0.8981        0.3514  1.1789
      2        0.3163       0.9166        0.2875  1.1827
      3        0.2642       0.9271        0.2480  1.1806
      4        0.2273       0.9350        0.2185  1.1822
      5        0.1997       0.9428        0.1960  1.1949
      6        0.1776       0.9488        0.1787  1.1904
      7        0.1597       0.9525        0.1648  1.2071
      8        0.1449       0.9567        0.1535  1.2491
      9        0.1324       0.9599        0.1441  1.2166
     10        0.1218       0.9616        0.1366  1.2151
     11        0.1128       0.9633        0.1301  1.2167
     12        0.1049       0.9644        0.1248  1.2613
     13        0.0979       0.9657        0.1201  1.2944
     14        0.0918       0.9665        0.1160  1.2566
     15        0.0864       0.9678        0.1125  1.2903
     16        0.0815       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=784, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  ),
)

In [7]:
#모형의 정확도 계산
import numpy as np
pred = net.predict(X_test)
y_test = y_test.cpu().numpy()
accuracy = np.mean(pred == y_test)
accuracy

0.9718

In [8]:
from torchinfo import summary
summary(model, input_size=(100,784))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [100, 10]                 --
├─Linear: 1-1                            [100, 128]                100,480
├─Linear: 1-2                            [100, 10]                 1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
Total mult-adds (M): 10.18
Input size (MB): 0.31
Forward/backward pass size (MB): 0.11
Params size (MB): 0.41
Estimated Total Size (MB): 0.83